# Support Vector Machines (SVM) - Linear Kernel

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [4]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(   
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=LinearSVC(random_state=42), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=6)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f84fb211c70>>)
Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f11ca9e7c70>>)
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fb7190c3c70>>)


/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-CNZVnMpw/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fb7190c3e20>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(3, 8))


/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-CNZVnMpw/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-CNZVnMpw/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-CNZVnMpw/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[('CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f11ca9e7c70>>)', f1_macro           0.815784
recall_macro       0.817388
precision_macro    0.822525
accuracy           0.816530
auc_score          0.901381
dtype: float64), ('CountVectorizer(ngram_range=(1, 3),\n                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f84fb211c70>>)', f1_macro           0.777818
recall_macro       0.785446
precision_macro    0.813755
accuracy           0.783550
auc_score          0.905550
dtype: float64), ("CountVectorizer(analyzer='char', ngram_range=(1, 5))", f1_macro           0.807817
recall_macro       0.809510
precision_macro    0.815091
accuracy           0.808720
auc_score          0.896197
dtype: float64), ("CountVectorizer(analyzer='char', ngram_range=(4, 5))", f1_macro           0.785534
recall_macro       0.788530
precision_macro    0.797879
accuracy           0.

In [5]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/linear_svc.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.815784,0.817388,0.822525,0.816530,0.901381,CountVectorizer(tokenizer=<bound method TweetT...
1,0.777818,0.785446,0.813755,0.783550,0.905550,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.807817,0.809510,0.815091,0.808720,0.896197,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.785534,0.788530,0.797879,0.787342,0.879297,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.769784,0.774988,0.791697,0.773716,0.876239,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.866946,0.867229,0.867810,0.867064,0.936061,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.852653,0.853298,0.858306,0.853422,0.931513,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.853453,0.853732,0.854208,0.853556,0.927545,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.824912,0.825852,0.828425,0.825375,0.905029,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.825918,0.826255,0.827165,0.826105,0.904006,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [6]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.866946
recall_macro                                                0.867229
precision_macro                                              0.86781
accuracy                                                    0.867064
auc_score                                                   0.936061
vectorizer         TfidfVectorizer(tokenizer=<bound method TweetT...
Name: 5, dtype: object

Vectorizer: TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fb7190c3c70>>)


In [7]:
pd.read_csv("../../results/linear_svc.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.815784,0.817388,0.822525,0.816530,0.901381,CountVectorizer(tokenizer=<bound method TweetT...
1,0.777818,0.785446,0.813755,0.783550,0.905550,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.807817,0.809510,0.815091,0.808720,0.896197,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.785534,0.788530,0.797879,0.787342,0.879297,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.769784,0.774988,0.791697,0.773716,0.876239,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.866946,0.867229,0.867810,0.867064,0.936061,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.852653,0.853298,0.858306,0.853422,0.931513,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.853453,0.853732,0.854208,0.853556,0.927545,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.824912,0.825852,0.828425,0.825375,0.905029,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.825918,0.826255,0.827165,0.826105,0.904006,"TfidfVectorizer(analyzer='char', ngram_range=(..."
